<a href="https://colab.research.google.com/github/awaaat/Machine_learning-Deep_learning/blob/main/Quick_Tensorflow_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")

Tensorflow version: 2.15.0


In [14]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [15]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255.0, X_test/255.0
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

X_train: (60000, 28, 28)
y_train: (60000,)
X_test: (10000, 28, 28)
y_test: (10000,)


In [16]:
#We then add channel dimensions
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")

In [17]:
training_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(32)
testing_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)


#We are now going to be building the tf.keras model using the Keras model subclassing API:

In [18]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation = "relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation = "relu")
    self.d2 = Dense(10)

  def call(self, X):
    X = self.conv1(X)
    X = X = self.flatten(X)
    X = self.d1(X)
    return self.d2(X)


model = MyModel()

In [19]:
#We then Choose an optimizer and loss function for training:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam()

#Let us select performannce evaluation meausres of the model
train_loss = tf.keras.metrics.Mean(name  = "training loss")
train_accurracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "training accuracy")

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accurracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [20]:
#We noe use the Use tf.GradientTape to train the model
@tf.function
def training_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training = True)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))


  train_loss(loss)
  train_accurracy(labels, predictions)

In [21]:
#Let us have the testing step
@tf.function
def testing_step(images, labels):
  predictions = model(images, training = False)
  t_loss = loss_fn(labels, predictions)
  test_loss(t_loss)
  test_accurracy(labels, predictions)

In [ ]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_state()
  train_accurracy.reset_state()
  test_loss.reset_state()
  test_accurracy.reset_state()


  #Apply the training step
  for images, labels in training_ds:
    training_step(images, labels)
  for test_images, test_labels in testing_ds:
    testing_step(test_images, test_labels)
  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accurracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accurracy.result() * 100:0.2f}'
  )

Epoch 1, Loss: 0.13, Accuracy: 95.88, Test Loss: 0.08, Test Accuracy: 97.43
